In [57]:
import pandas as pd
import glob
import json
import pickle
import numpy as np

In [2]:
# read metadata
metadata = pd.read_csv('./CovidData/metadata.csv', dtype={
    'pubmed_id': str, 
    'Microsoft Academic Paper ID': str, 'doi': str
})

/gscratch/stf/nelljy/tools/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,5,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(metadata.head())

   cord_uid                                       sha source_x  \
0  ug7v899j  d1aafb70c066a2068b02786f8929fd9c900897fb      PMC   
1  02tnwd4m  6b0567729c2143a66d737eb0a2f63f2dce2e5a7d      PMC   
2  ejv2xln0  06ced00a5fc04215949aa72528f2eeaae1d58927      PMC   
3  2b73a28n  348055649b6b8cf2b9a376498df9bf41f7123605      PMC   
4  9785vg6d  5f48792a5fa08bed9f56016f4981ae2ca6031b32      PMC   

                                               title                    doi  \
0  Clinical features of culture-proven Mycoplasma...  10.1186/1471-2334-1-6   
1  Nitric oxide: a pro-inflammatory mediator in l...           10.1186/rr14   
2    Surfactant protein-D and pulmonary host defense           10.1186/rr19   
3               Role of endothelin-1 in lung disease           10.1186/rr44   
4  Gene expression in epithelial cells in respons...           10.1186/rr61   

      pmcid pubmed_id license  \
0  PMC35282  11472636   no-cc   
1  PMC59543  11667967   no-cc   
2  PMC59549  11667972   no-cc

In [4]:
# the number of papers
len(metadata)

740302

In [5]:
# the percent of null values in each column
metadata.isnull().sum()/len(metadata)

cord_uid            0.000000
sha                 0.686875
source_x            0.000000
title               0.000493
doi                 0.504897
pmcid               0.675089
pubmed_id           0.571102
license             0.000000
abstract            0.261919
publish_time        0.000307
authors             0.023066
journal             0.061969
mag_id              1.000000
who_covidence_id    0.525287
arxiv_id            0.987713
pdf_json_files      0.686875
pmc_json_files      0.743873
url                 0.472467
s2_id               0.076933
dtype: float64

In [6]:
# the number of unique cord_uid
# cord_uid: this persistent identifier is a 8-char alphanumeric string unique to each entry
metadata.cord_uid.nunique()

659301

In [7]:
# the number of unique sha
# sha: the hash of the PDFs are in 'sha'
metadata.sha.nunique()

231780

In [8]:
# the number of unique titles
metadata.title.nunique()

554017

In [10]:
# json file path
all_json_path = glob.glob('./CovidData/document_parses/pdf_json/*.json')
len(all_json_path)

247236

In [11]:
with open(all_json_path[1]) as f:
    first_entry = json.load(f)
    print(json.dumps(first_entry, indent=4))

{
    "paper_id": "c40b7d23a6c005ab11aa812d9d43567140feedeb",
    "metadata": {
        "title": "Certifying Irreducibility in Z[x]",
        "authors": [
            {
                "first": "John",
                "middle": [],
                "last": "Abbott",
                "suffix": "",
                "affiliation": {
                    "laboratory": "",
                    "institution": "Universit\u00e4t Passau",
                    "location": {
                        "settlement": "Passau",
                        "country": "Germany"
                    }
                },
                "email": "abbott@dima.unige.it"
            }
        ]
    },
    "abstract": [
        {
            "text": "We consider the question of certifying that a polynomial in",
            "cite_spans": [],
            "ref_spans": [],
            "section": "Abstract"
        },
        {
            "text": "Knowing that a polynomial is irreducible lets us recognise that a quotient rin

In [12]:
with open(all_json_path[0]) as f:
    item = json.load(f)
    for i in item['abstract']:
        print(i['text'])

In [13]:
# write a class to read information from json files
class filereader:
    def __init__(self, file_path):
        with open(file_path) as f:
            content = json.load(f)
            self.paper_id = content['paper_id']
            
            self.abstract = []
            self.body_text = []
            
            # get abstract if the abstract is not none
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
                
            
            # get body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            
            # join abstract into one string
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)

In [14]:
filereader(all_json_path[0]).paper_id

'166847712b1c14b3864156a815a4613769821c31'

In [15]:
filereader(all_json_path[0]).abstract

''

In [16]:
filereader(all_json_path[0]).body_text

'Coronavirus disease 2019 , caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), has become a global health threat, infecting 27, 150, 797 cases 1, 844, 863 people and resulting in 889, 256 deaths 117, 021 deaths at the time of the last writing (September 7th, 2020). This new type of respiratory illness is characterized by rapid human-to-human transmission, having achieved pandemic spread (1) . There are currently no therapeutics or vaccines available and no pre-existing immunity in the population.\nBecause COVID-19 is a new disease, our awareness and knowledge are gradually increasing based on ongoing research findings and clinical practice experience. A significant amount of collective work has been done to understand the prevention and treatment strategies of COVID-19. The disease has posed unparalleled challenges for health care communities, the general population, and in particular, for patients suffering from chronic diseases (such as: thalassemia and sickle ce

In [17]:
# load json data into dataframe
# json_dict = {'paper_id': [], 'abstract': [], 'body_text': []}
# for idx, file_path in enumerate(all_json_path):
#     if idx % (len(all_json_path) // 10) == 0:
#         print('Processing index: {} of {}'.format(idx, len(all_json_path)))
#     content = filereader(file_path)
#     json_dict['paper_id'].append(content.paper_id)
#     json_dict['abstract'].append(content.abstract)
#     json_dict['body_text'].append(content.body_text)

In [25]:
# with open('json_dict.pkl', 'wb') as f:
#     pickle.dump(json_dict, f)

In [18]:
## read pickle file
json_dict_pickle = pickle.load(open('json_dict.pkl', 'rb'))

In [19]:
json_df = pd.DataFrame(json_dict_pickle, columns=['paper_id', 'abstract', 'body_text'])
json_df.head()

,paper_id,abstract,body_text
0,166847712b1c14b3864156a815a4613769821c31,,"Coronavirus disease 2019 , caused by severe ac..."
1,c40b7d23a6c005ab11aa812d9d43567140feedeb,We consider the question of certifying that a ...,A certificate that object X has property P is ...
2,f6e7d1aaef6479469705a2ae0ea9a5e5a5e18abf,Clinical and experimental preparations of IgG/...,"Immune complex, as a form of autoimmune disord..."
3,75fdfca28670826b0b6448f28d71403add61bec8,,Coronavirus disease 2019 infection is evolving...
4,cba5fe8d72683af2fdc9f72d7252636915f86c2c,The growing threat of an influenza pandemic pr...,T he growing threat of a pandemic caused by in...


In [20]:
json_df.shape

(247236, 3)

In [114]:
# merge json_df with metadata
data = pd.merge(json_df, metadata, left_on='paper_id', right_on='sha', how='left').drop('sha', axis=1)

In [115]:
data.columns

Index(['paper_id', 'abstract_x', 'body_text', 'cord_uid', 'source_x', 'title',
       'doi', 'pmcid', 'pubmed_id', 'license', 'abstract_y', 'publish_time',
       'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id',
       'pdf_json_files', 'pmc_json_files', 'url', 's2_id'],
      dtype='object')

In [116]:
# read pmc_json files
all_pmc_path = glob.glob('./CovidData/document_parses/pmc_json/*.json')

In [117]:
# write a class to read information from pmc json files
# It contains no abstracts
class filereader:
    def __init__(self, file_path):
        with open(file_path) as f:
            content = json.load(f)
            self.paper_id = content['paper_id']
            
            self.body_text = []
                  
            # get body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            
            # join abstract into one string
            self.body_text = '\n'.join(self.body_text)

In [118]:
# pmc_dict = {'paper_id': [], 'body_text': []}
# for idx, file_path in enumerate(all_pmc_path):
#     if idx % (len(all_pmc_path) // 10) == 0:
#         print('Processing index: {} of {}'.format(idx, len(all_pmc_path)))
#     content = filereader(file_path)
#     pmc_dict['paper_id'].append(content.paper_id)
#     pmc_dict['body_text'].append(content.body_text)

In [119]:
# with open('pmc_dict.pkl', 'wb') as f:
#     pickle.dump(pmc_dict, f)

In [120]:
pmc_dict_pickle = pickle.load(open('pmc_dict.pkl', 'rb'))

In [121]:
pmc_df = pd.DataFrame(pmc_dict_pickle, columns=['paper_id', 'body_text'])
pmc_df.head()

,paper_id,body_text
0,PMC3151229,"Since the beginning of the new millennium, epi..."
1,PMC8217165,"Syringe service programs (SSPs), which at mini..."
2,PMC7373225,Data from Johns Hopkins University updated tha...
3,PMC7361262,Australia is a food secure nation that exports...
4,PMC8068274,"In December 2019, health Chinese authorities r..."


In [122]:
# blank body text
pmc_df[pmc_df.body_text == ' ']

,paper_id,body_text
4155,PMC7300948,
7770,PMC8239874,
10602,PMC7536999,
37651,PMC7300792,
38243,PMC7267251,
39137,PMC7537237,
48512,PMC7300721,
58544,PMC8239713,
59017,PMC8018358,
59353,PMC7300741,


In [123]:
pmc_df = pmc_df[pmc_df.body_text != ' ']

In [124]:
data.head()

,paper_id,abstract_x,body_text,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,...,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,166847712b1c14b3864156a815a4613769821c31,,"Coronavirus disease 2019 , caused by severe ac...",cn16aj61,Medline; PMC,The Multifacets of COVID-19 in Adult Patients:...,10.23750/abm.v91i4.10665,PMC7927469,33525226,cc-by-nc-sa,...,2020-11-10,"Canatan, Duran; Vives Corrons, Joan Lluis; de ...",Acta Biomed,NaN,NaN,NaN,document_parses/pdf_json/166847712b1c14b386415...,document_parses/pmc_json/PMC7927469.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33525226/;...,231758303.0
1,c40b7d23a6c005ab11aa812d9d43567140feedeb,We consider the question of certifying that a ...,A certificate that object X has property P is ...,wtjz00bq,PMC,Certifying Irreducibility in [Formula: see text],10.1007/978-3-030-52200-1_46,PMC7340962,NaN,no-cc,...,2020-06-06,"Abbott, John",Mathematical Software - ICMS 2020,NaN,NaN,NaN,document_parses/pdf_json/c40b7d23a6c005ab11aa8...,document_parses/pmc_json/PMC7340962.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
2,f6e7d1aaef6479469705a2ae0ea9a5e5a5e18abf,Clinical and experimental preparations of IgG/...,"Immune complex, as a form of autoimmune disord...",tlfzjmju,Medline; PMC,Immunoregulatory functions of immune complexes...,10.15252/emmm.201606593,PMC5048363,27572622,cc-by,...,2016-08-29,"Wen, Yu‐mei; Mu, Libing; Shi, Yan",EMBO Mol Med,NaN,NaN,NaN,document_parses/pdf_json/f6e7d1aaef6479469705a...,document_parses/pmc_json/PMC5048363.xml.json,https://doi.org/10.15252/emmm.201606593; https...,17088715.0
3,75fdfca28670826b0b6448f28d71403add61bec8,,Coronavirus disease 2019 infection is evolving...,5t8kp4m1,Medline; PMC,Excessive and Problematic Internet Use During ...,10.3389/fpubh.2020.609347,PMC7773910,33392143,cc-by,...,2020-12-17,"Kawabe, Kentaro; Hosokawa, Rie; Nakachi, Kiwam...",Front Public Health,NaN,NaN,NaN,document_parses/pdf_json/75fdfca28670826b0b644...,document_parses/pmc_json/PMC7773910.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33392143/;...,229290856.0
4,cba5fe8d72683af2fdc9f72d7252636915f86c2c,The growing threat of an influenza pandemic pr...,T he growing threat of a pandemic caused by in...,rhbqgjnm,Medline; PMC,Viable Viral Efficiency of N95 and P100 Respir...,10.1080/15459624.2013.818228,PMC7196694,24011377,no-cc,...,2013-09-06,"Gardner, Paul D.; Eshbaugh, Jonathan P.; Harpe...",J Occup Environ Hyg,NaN,NaN,NaN,document_parses/pdf_json/cba5fe8d72683af2fdc9f...,document_parses/pmc_json/PMC7196694.xml.json,https://doi.org/10.1080/15459624.2013.818228; ...,37176561.0


In [125]:
# merge pmc with data
data = pd.merge(data, pmc_df, left_on='pmcid', right_on='paper_id', how='left').drop('paper_id_y', axis=1)
data.head()

,paper_id_x,abstract_x,body_text_x,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,...,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text_y
0,166847712b1c14b3864156a815a4613769821c31,,"Coronavirus disease 2019 , caused by severe ac...",cn16aj61,Medline; PMC,The Multifacets of COVID-19 in Adult Patients:...,10.23750/abm.v91i4.10665,PMC7927469,33525226,cc-by-nc-sa,...,"Canatan, Duran; Vives Corrons, Joan Lluis; de ...",Acta Biomed,NaN,NaN,NaN,document_parses/pdf_json/166847712b1c14b386415...,document_parses/pmc_json/PMC7927469.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33525226/;...,231758303.0,"Coronavirus disease 2019 (COVID-19), caused by..."
1,c40b7d23a6c005ab11aa812d9d43567140feedeb,We consider the question of certifying that a ...,A certificate that object X has property P is ...,wtjz00bq,PMC,Certifying Irreducibility in [Formula: see text],10.1007/978-3-030-52200-1_46,PMC7340962,NaN,no-cc,...,"Abbott, John",Mathematical Software - ICMS 2020,NaN,NaN,NaN,document_parses/pdf_json/c40b7d23a6c005ab11aa8...,document_parses/pmc_json/PMC7340962.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,We can certify that a positive integer n is pr...
2,f6e7d1aaef6479469705a2ae0ea9a5e5a5e18abf,Clinical and experimental preparations of IgG/...,"Immune complex, as a form of autoimmune disord...",tlfzjmju,Medline; PMC,Immunoregulatory functions of immune complexes...,10.15252/emmm.201606593,PMC5048363,27572622,cc-by,...,"Wen, Yu‐mei; Mu, Libing; Shi, Yan",EMBO Mol Med,NaN,NaN,NaN,document_parses/pdf_json/f6e7d1aaef6479469705a...,document_parses/pmc_json/PMC5048363.xml.json,https://doi.org/10.15252/emmm.201606593; https...,17088715.0,"Immune complex, as a form of autoimmune disord..."
3,75fdfca28670826b0b6448f28d71403add61bec8,,Coronavirus disease 2019 infection is evolving...,5t8kp4m1,Medline; PMC,Excessive and Problematic Internet Use During ...,10.3389/fpubh.2020.609347,PMC7773910,33392143,cc-by,...,"Kawabe, Kentaro; Hosokawa, Rie; Nakachi, Kiwam...",Front Public Health,NaN,NaN,NaN,document_parses/pdf_json/75fdfca28670826b0b644...,document_parses/pmc_json/PMC7773910.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33392143/;...,229290856.0,Coronavirus disease 2019 (COVID-19) infection ...
4,cba5fe8d72683af2fdc9f72d7252636915f86c2c,The growing threat of an influenza pandemic pr...,T he growing threat of a pandemic caused by in...,rhbqgjnm,Medline; PMC,Viable Viral Efficiency of N95 and P100 Respir...,10.1080/15459624.2013.818228,PMC7196694,24011377,no-cc,...,"Gardner, Paul D.; Eshbaugh, Jonathan P.; Harpe...",J Occup Environ Hyg,NaN,NaN,NaN,document_parses/pdf_json/cba5fe8d72683af2fdc9f...,document_parses/pmc_json/PMC7196694.xml.json,https://doi.org/10.1080/15459624.2013.818228; ...,37176561.0,The growing threat of a pandemic caused by inf...


In [126]:
print(data.shape)
data.drop_duplicates(['body_text_x', 'body_text_y'], inplace=True)
print(data.shape)

(247263, 22)
(244754, 22)


In [127]:
# abstract_x is from json files, abstract_y is from metadata
# a large proportion of data have different abstract
data[data.abstract_x != data.abstract_y].shape[0] / data.shape[0]

0.9102445721009667

In [128]:
data[data.abstract_x != data.abstract_y][['abstract_x', 'abstract_y', 'url']].head()

,abstract_x,abstract_y,url
0,,COVID-19 is an infectious disease caused by th...,https://www.ncbi.nlm.nih.gov/pubmed/33525226/;...
1,We consider the question of certifying that a ...,We consider the question of certifying that a ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
2,Clinical and experimental preparations of IgG/...,Clinical and experimental preparations of IgG/...,https://doi.org/10.15252/emmm.201606593; https...
3,,Internet use in the youth has increased manifo...,https://www.ncbi.nlm.nih.gov/pubmed/33392143/;...
5,sacrament that has also adapted and has adopte...,The Covid-19 pandemic has drastically changed ...,https://www.ncbi.nlm.nih.gov/pubmed/34080622/;...


In [129]:
data[data.abstract_x != data.abstract_y][['abstract_x', 'abstract_y', 'url']].tail(3)

,abstract_x,abstract_y,url
247258,"Citation: Zhang, B.; Zou, J.; Zhang, Q.; Wang,...",Eukaryotic elongation factor 2 kinase (eEF2K o...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
247260,,NaN,https://doi.org/10.3928/24748307-20210504-01; ...
247262,Relatively limited data are available regardin...,NaN,NaN


In [130]:
# For e.g., the abstract in json file is not correct, while the abstract in metadata is correct

In [131]:
data[(data.abstract_x != data.abstract_y) &
     (data.abstract_y.isnull())][['abstract_x', 'abstract_y', 'url', 'body_text_x', 'body_text_y']]

,abstract_x,abstract_y,url,body_text_x,body_text_y
8,,NaN,https://www.ncbi.nlm.nih.gov/pubmed/33005947/;...,"In many ways, it seems to me that the COVID-19...","In many ways, it seems to me that the COVID-19..."
12,,NaN,https://www.sciencedirect.com/science/article/...,La crise sanitaire liée au SARS-Coronavirus-19...,NaN
16,Purpose: Numerous publications during the COVI...,NaN,NaN,The coronavirus (SARS-CoV-2) outbreak was firs...,NaN
17,,NaN,NaN,Some intentions in the SAP will be reported la...,NaN
21,Ibrutinib is the first covalent inhibitor of B...,NaN,NaN,Covalent inhibitor drug discovery has re-emerg...,NaN
...,...,...,...,...,...
247247,,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,"In March 2009, an outbreak of H1N1 influenza A...","In March 2009, an outbreak of H1N1 influenza A..."
247250,,NaN,https://api.elsevier.com/content/article/pii/S...,"El nuevo coronavirus, que comenzó como un brot...",NaN
247254,"Nowadays, over 200 countries face a wellbeing ...",NaN,NaN,The recently declared global emergency followi...,NaN
247260,,NaN,https://doi.org/10.3928/24748307-20210504-01; ...,"The 2020 Integrating Research, Practice and Po...","The 2020 Integrating Research, Practice and Po..."


In [132]:
# I check the first row, the paper is a letter and doesn't have abstract.
print(data.shape, data.abstract_x.isnull().sum(), data[data.abstract_x == ''].shape, data[data.abstract_x == ' '].shape)
print(data.shape, data.abstract_y.isnull().sum(), data[data.abstract_y == ''].shape, data[data.abstract_y == ' '].shape)

(244754, 22) 0 (74439, 22) (4, 22)
(244754, 22) 60055 (0, 22) (0, 22)


In [133]:
# the abstract in metadata seems to be more reliable, so we use these, and fill the missing values in abstract_y 
# with the abstract_x
data.loc[data.abstract_y.isnull() & 
         (data.abstract_x != '') & 
         (data.abstract_x != ' ') & 
         (~data.abstract_x.isnull()), 'abstract_y'] = data[data.abstract_y.isnull() &
                                                           (data.abstract_x != '') & 
                                                           (data.abstract_x != ' ') & 
                                                           (~data.abstract_x.isnull())].abstract_x

In [134]:
data[['abstract_x', 'abstract_y']].tail(10)

,abstract_x,abstract_y
247253,,This study explored the internal and external ...
247254,"Nowadays, over 200 countries face a wellbeing ...","Nowadays, over 200 countries face a wellbeing ..."
247255,"The airborne dynamics of respiratory droplets,...","The airborne dynamics of respiratory droplets,..."
247256,,This chapter sets the scene for this book. It ...
247257,The expansion of Covid-19 has severely hit the...,The expansion of Covid-19 has severely hit the...
247258,"Citation: Zhang, B.; Zou, J.; Zhang, Q.; Wang,...",Eukaryotic elongation factor 2 kinase (eEF2K o...
247259,Viruses could manipulate cellular machinery to...,Viruses could manipulate cellular machinery to...
247260,,NaN
247261,Aptamers have a promising role in the field of...,Aptamers have a promising role in the field of...
247262,Relatively limited data are available regardin...,Relatively limited data are available regardin...


In [135]:
(data.abstract_y.isnull() & (data.abstract_x != '') & (data.abstract_x != ' ')).sum()

0

In [136]:
data.rename(columns={'abstract_y': 'abstract'}, inplace=True)
data.drop('abstract_x', axis=1, inplace=True)
data.head()

,paper_id_x,body_text_x,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,...,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text_y
0,166847712b1c14b3864156a815a4613769821c31,"Coronavirus disease 2019 , caused by severe ac...",cn16aj61,Medline; PMC,The Multifacets of COVID-19 in Adult Patients:...,10.23750/abm.v91i4.10665,PMC7927469,33525226,cc-by-nc-sa,COVID-19 is an infectious disease caused by th...,...,"Canatan, Duran; Vives Corrons, Joan Lluis; de ...",Acta Biomed,NaN,NaN,NaN,document_parses/pdf_json/166847712b1c14b386415...,document_parses/pmc_json/PMC7927469.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33525226/;...,231758303.0,"Coronavirus disease 2019 (COVID-19), caused by..."
1,c40b7d23a6c005ab11aa812d9d43567140feedeb,A certificate that object X has property P is ...,wtjz00bq,PMC,Certifying Irreducibility in [Formula: see text],10.1007/978-3-030-52200-1_46,PMC7340962,NaN,no-cc,We consider the question of certifying that a ...,...,"Abbott, John",Mathematical Software - ICMS 2020,NaN,NaN,NaN,document_parses/pdf_json/c40b7d23a6c005ab11aa8...,document_parses/pmc_json/PMC7340962.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,We can certify that a positive integer n is pr...
2,f6e7d1aaef6479469705a2ae0ea9a5e5a5e18abf,"Immune complex, as a form of autoimmune disord...",tlfzjmju,Medline; PMC,Immunoregulatory functions of immune complexes...,10.15252/emmm.201606593,PMC5048363,27572622,cc-by,Clinical and experimental preparations of IgG/...,...,"Wen, Yu‐mei; Mu, Libing; Shi, Yan",EMBO Mol Med,NaN,NaN,NaN,document_parses/pdf_json/f6e7d1aaef6479469705a...,document_parses/pmc_json/PMC5048363.xml.json,https://doi.org/10.15252/emmm.201606593; https...,17088715.0,"Immune complex, as a form of autoimmune disord..."
3,75fdfca28670826b0b6448f28d71403add61bec8,Coronavirus disease 2019 infection is evolving...,5t8kp4m1,Medline; PMC,Excessive and Problematic Internet Use During ...,10.3389/fpubh.2020.609347,PMC7773910,33392143,cc-by,Internet use in the youth has increased manifo...,...,"Kawabe, Kentaro; Hosokawa, Rie; Nakachi, Kiwam...",Front Public Health,NaN,NaN,NaN,document_parses/pdf_json/75fdfca28670826b0b644...,document_parses/pmc_json/PMC7773910.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33392143/;...,229290856.0,Coronavirus disease 2019 (COVID-19) infection ...
4,cba5fe8d72683af2fdc9f72d7252636915f86c2c,T he growing threat of a pandemic caused by in...,rhbqgjnm,Medline; PMC,Viable Viral Efficiency of N95 and P100 Respir...,10.1080/15459624.2013.818228,PMC7196694,24011377,no-cc,The growing threat of an influenza pandemic pr...,...,"Gardner, Paul D.; Eshbaugh, Jonathan P.; Harpe...",J Occup Environ Hyg,NaN,NaN,NaN,document_parses/pdf_json/cba5fe8d72683af2fdc9f...,document_parses/pmc_json/PMC7196694.xml.json,https://doi.org/10.1080/15459624.2013.818228; ...,37176561.0,The growing threat of a pandemic caused by inf...


In [137]:
# body_text_x is from json files, body_text_y is from pmc files
# most of the data have different body text
data[data.body_text_x != data.body_text_y].shape[0] / data.shape[0]

0.9999877427948062

In [138]:
# similar body_text
data[data.body_text_x != data.body_text_y][['body_text_x', 'body_text_y', 'url']]

,body_text_x,body_text_y,url
0,"Coronavirus disease 2019 , caused by severe ac...","Coronavirus disease 2019 (COVID-19), caused by...",https://www.ncbi.nlm.nih.gov/pubmed/33525226/;...
1,A certificate that object X has property P is ...,We can certify that a positive integer n is pr...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
2,"Immune complex, as a form of autoimmune disord...","Immune complex, as a form of autoimmune disord...",https://doi.org/10.15252/emmm.201606593; https...
3,Coronavirus disease 2019 infection is evolving...,Coronavirus disease 2019 (COVID-19) infection ...,https://www.ncbi.nlm.nih.gov/pubmed/33392143/;...
4,T he growing threat of a pandemic caused by in...,The growing threat of a pandemic caused by inf...,https://doi.org/10.1080/15459624.2013.818228; ...
...,...,...,...
247258,Targeted therapy is an important strategy for ...,Targeted therapy is an important strategy for ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
247259,Invasive viruses are adaptively infectious and...,Invasive viruses are adaptively infectious and...,https://www.ncbi.nlm.nih.gov/pubmed/23162545/;...
247260,"The 2020 Integrating Research, Practice and Po...","The 2020 Integrating Research, Practice and Po...",https://doi.org/10.3928/24748307-20210504-01; ...
247261,Aptamers can be broadly defined as short biomo...,Aptamers can be broadly defined as short biomo...,https://doi.org/10.3390/molecules201219739; ht...


In [139]:
# use the text from pmc file (body_text_y) trusting the statement that it's of higher quality
print(data.body_text_x.isnull().sum(), data[data.body_text_x == ''].shape, data[data.body_text_x == ' '].shape)
print(data.body_text_y.isnull().sum(), data[data.body_text_y == ''].shape, data[data.body_text_y == ' '].shape)

0 (0, 21) (0, 21)
79128 (0, 21) (0, 21)


In [140]:
data.loc[data.body_text_y.isnull(), 'body_text_y'] = data[data.body_text_y.isnull()].body_text_x
data.body_text_y.isnull().sum()

0

In [141]:
data.rename(columns = {'body_text_y': 'body_text'}, inplace=True)
data.drop('body_text_x', axis=1, inplace=True)
data.head(10)

,paper_id_x,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text
0,166847712b1c14b3864156a815a4613769821c31,cn16aj61,Medline; PMC,The Multifacets of COVID-19 in Adult Patients:...,10.23750/abm.v91i4.10665,PMC7927469,33525226,cc-by-nc-sa,COVID-19 is an infectious disease caused by th...,2020-11-10,"Canatan, Duran; Vives Corrons, Joan Lluis; de ...",Acta Biomed,NaN,NaN,NaN,document_parses/pdf_json/166847712b1c14b386415...,document_parses/pmc_json/PMC7927469.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33525226/;...,231758303.0,"Coronavirus disease 2019 (COVID-19), caused by..."
1,c40b7d23a6c005ab11aa812d9d43567140feedeb,wtjz00bq,PMC,Certifying Irreducibility in [Formula: see text],10.1007/978-3-030-52200-1_46,PMC7340962,NaN,no-cc,We consider the question of certifying that a ...,2020-06-06,"Abbott, John",Mathematical Software - ICMS 2020,NaN,NaN,NaN,document_parses/pdf_json/c40b7d23a6c005ab11aa8...,document_parses/pmc_json/PMC7340962.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,We can certify that a positive integer n is pr...
2,f6e7d1aaef6479469705a2ae0ea9a5e5a5e18abf,tlfzjmju,Medline; PMC,Immunoregulatory functions of immune complexes...,10.15252/emmm.201606593,PMC5048363,27572622,cc-by,Clinical and experimental preparations of IgG/...,2016-08-29,"Wen, Yu‐mei; Mu, Libing; Shi, Yan",EMBO Mol Med,NaN,NaN,NaN,document_parses/pdf_json/f6e7d1aaef6479469705a...,document_parses/pmc_json/PMC5048363.xml.json,https://doi.org/10.15252/emmm.201606593; https...,17088715.0,"Immune complex, as a form of autoimmune disord..."
3,75fdfca28670826b0b6448f28d71403add61bec8,5t8kp4m1,Medline; PMC,Excessive and Problematic Internet Use During ...,10.3389/fpubh.2020.609347,PMC7773910,33392143,cc-by,Internet use in the youth has increased manifo...,2020-12-17,"Kawabe, Kentaro; Hosokawa, Rie; Nakachi, Kiwam...",Front Public Health,NaN,NaN,NaN,document_parses/pdf_json/75fdfca28670826b0b644...,document_parses/pmc_json/PMC7773910.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33392143/;...,229290856.0,Coronavirus disease 2019 (COVID-19) infection ...
4,cba5fe8d72683af2fdc9f72d7252636915f86c2c,rhbqgjnm,Medline; PMC,Viable Viral Efficiency of N95 and P100 Respir...,10.1080/15459624.2013.818228,PMC7196694,24011377,no-cc,The growing threat of an influenza pandemic pr...,2013-09-06,"Gardner, Paul D.; Eshbaugh, Jonathan P.; Harpe...",J Occup Environ Hyg,NaN,NaN,NaN,document_parses/pdf_json/cba5fe8d72683af2fdc9f...,document_parses/pmc_json/PMC7196694.xml.json,https://doi.org/10.1080/15459624.2013.818228; ...,37176561.0,The growing threat of a pandemic caused by inf...
5,baf277701767e430f0260fa6a28780a9cd093fb8,al4t7uyf,Medline; PMC,The sacrament of confession during Covid-19 pa...,10.1093/pubmed/fdab193,PMC8194966,34080622,no-cc,The Covid-19 pandemic has drastically changed ...,2021-06-03,"Macaraan, Willard Enrique R",J Public Health (Oxf),NaN,NaN,NaN,document_parses/pdf_json/baf277701767e430f0260...,document_parses/pmc_json/PMC8194966.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34080622/;...,235320614.0,The Covid-19 pandemic has essentially changed ...
6,c5d77f7c8eaf8af4066e3ade6d4de2898583951a,q5zh5ixx,Elsevier; Medline; PMC,Spatial variation of dissolved organic nitroge...,10.1016/j.watres.2021.117138,PMC8036133,33895589,no-cc,Intensified sanitization practices during the ...,2021-04-11,"Wang, Leyun; Zhang, Xian; Chen, Shanshan; Meng...",Water Res,NaN,NaN,NaN,document_parses/pdf_json/c5d77f7c8eaf8af4066e3...,document_parses/pmc_json/PMC8036133.xml.json,https://api.elsevier.com/content/article/pii/S...,233202327.0,Dissolved organic nitrogen (DON) refers to the...
7,efd04c4e2388b36df41975af30c0fad9c3accb1a,xd7x393g,Medline; PMC,CD8 T cells compensate for impaired humoral im...,10.21203/rs.3.rs-162289/v1,PMC7872363,33564756,cc-by,Cancer patients have increased morbidity and m...,2021-02-02,"Huang, Alexander; Bange, Erin; Han, Ni

In [142]:
data.rename(columns={'paper_id_x': 'paper_id', 'source_x': 'source'}, inplace=True)

In [143]:
# duplicate paper id
data.paper_id.nunique() / data.shape[0]

0.9999550569142894

In [146]:
# check their body text, luckly the body text are the same
data[data.duplicated(subset='paper_id', keep=False)]

,paper_id,cord_uid,source,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text
29112,dcec56399a34e7323e4729f10d9c8363c59c1ba1,6eae03m5,ArXiv,EpiCovDA: a mechanistic COVID-19 forecasting m...,NaN,NaN,NaN,arxiv,We introduce a minimalist outbreak forecasting...,2021-05-12,"Biegel, Hannah R.; Lega, Joceline",NaN,NaN,NaN,2105.05471,document_parses/pdf_json/dcec56399a34e7323e472...,NaN,https://arxiv.org/pdf/2105.05471v1.pdf,234470142.0,"the other hand, lack of knowledge about sensit..."
29113,dcec56399a34e7323e4729f10d9c8363c59c1ba1,5y9di2cb,Medline; PMC,EpiCovDA: a mechanistic COVID-19 forecasting m...,NaN,PMC8132228,34012991,cc-by-nc-nd,We introduce a minimalist outbreak forecasting...,2021-05-12,"Biegel, Hannah R.; Lega, Joceline",ArXiv,NaN,NaN,NaN,document_parses/pdf_json/dcec56399a34e7323e472...,document_parses/pmc_json/PMC8132228.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34012991/,234470142.0,We have developed a novel and computationally ...
81233,2196c1b6ef3f7a2ea7dd7a9ae08cd6e72369a648,6vpmbnmh,ArXiv,Bam-readcount -- rapid generation of basepair-...,NaN,NaN,NaN,arxiv,Bam-readcount is a utility for generating low-...,2021-07-27,"Khanna, Ajay; Larson, David E.; Srivatsan, Sri...",NaN,NaN,NaN,2107.12817,document_parses/pdf_json/2196c1b6ef3f7a2ea7dd7...,NaN,https://arxiv.org/pdf/2107.12817v1.pdf,236447791.0,Though many tools exist that can call simple g...
81234,2196c1b6ef3f7a2ea7dd7a9ae08cd6e72369a648,mvu3cz50,Medline; PMC,Bam-readcount - rapid generation of basepair-r...,NaN,PMC8328062,34341766,cc-by,Bam-readcount is a utility for generating low-...,2021-07-27,"Khanna, Ajay; Larson, David E.; Srivatsan, Sri...",ArXiv,NaN,NaN,NaN,document_parses/pdf_json/2196c1b6ef3f7a2ea7dd7...,document_parses/pmc_json/PMC8328062.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34341766/,236447791.0,Though many tools exist that can call simple g...
97138,36e2047d1674c3095617f3eb97f9f61e48989dfe,zhw8vh3e,PMC,Artificial Intelligence (AI) applications for ...,10.1016/j.dsx.2020.04.012,PMC7195043,32305024,no-cc,BACKGROUND AND AIMS: Healthcare delivery requi...,2020-04-14,"Vaishya, Raju; Javaid, Mohd; Khan, Ibrahim Hal...",Diabetes Metab Syndr,NaN,NaN,NaN,document_parses/pdf_json/36e2047d1674c3095617f...,document_parses/pmc_json/PMC7195043.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,"In this worldwide health crisis, the medical i..."
97139,36e2047d1674c3095617f3eb97f9f61e48989dfe,zhw8vh3e,Elsevier,Artificial Intelligence (AI) applications for ...,10.1016/j.dsx.2020.04.012,NaN,NaN,els-covid,Abstract Background and aims Healthcare delive...,2020-08-31,"Vaishya, Raju; Javaid, Mohd; Khan, Ibrahim Hal...",Diabetes & Metabolic Syndrome: Clinical Resear...,NaN,NaN,NaN,document_parses/pdf_json/36e2047d1674c3095617f...,NaN,https://api.elsevier.com/content/article/pii/S...,215819243.0,"In this worldwide health crisis, the medical i..."
117500,0ed3c6a5559cd73307184f51fc53ccc76da559bc,p92y5er2,Elsevier; Medline; PMC,Novel coronavirus disease (Covid-19): The firs...,10.1016/j.jinf.2020.02.020,PMC7127394,32119884,els-covid,NaN,2020-05-31,"Lillie, Patrick J.; Samson, Anda; Li, Ang; Ada...",Journal of Infection,NaN,NaN,NaN,document_parses/pdf_json/0ed3c6a5559cd73307184...,document_parses/pmc_json/PMC7127394.xml.json,https://api.elsevier.com/content/article/pii/S...,211832079.0,The authors declare that they have no known co...
117501,0ed3c6a5559cd73307184f51fc53ccc76da559bc,plwyjhl8,Elsevier; Medline; PMC,Simulating and forecasting the cumulative conf...,10.1016/j.jinf.2020.02.019,PMC7127674,32112887,els-covid,• Cumulative confirmed cases in China were wel...,2020-05-31,"Fu, Xinmiao; Ying, Qi; Zeng, Tieyong; Long, Ta...",Journal of Infection,NaN,NaN,NaN,document_parses/pdf_json/0ed3c6a5559cd73307184...,document_parses/pmc_json/PMC7127674.xml.json,https://doi.org/10.1016/j.jinf.2020.02.019; ht...,211573526.0,None.
117502,0ed3c6a5559cd73307184f51fc5

In [147]:
data.drop_duplicates(['paper_id'], inplace=True)
print(data[data.duplicated(subset='paper_id', keep=False)])

Empty DataFrame
Columns: [paper_id, cord_uid, source, title, doi, pmcid, pubmed_id, license, abstract, publish_time, authors, journal, mag_id, who_covidence_id, arxiv_id, pdf_json_files, pmc_json_files, url, s2_id, body_text]
Index: []


In [148]:
data.isnull().sum()

paper_id                 0
cord_uid             26922
source               26922
title                26926
doi                  35192
pmcid                51274
pubmed_id            77281
license              26922
abstract             37532
publish_time         26922
authors              29035
journal              49297
mag_id              244743
who_covidence_id    244743
arxiv_id            236500
pdf_json_files       26922
pmc_json_files       79119
url                  26922
s2_id                66039
body_text                0
dtype: int64